In [1]:
# MarkedLM 테스트 예제
#
# => input_ids : [CLS][MASK]문장[SEP]
# => attention_mask : 1111111111
# => token_type_ids : 0000000


import numpy as np
import torch
import torch.nn.functional as F
import os
import sys
from transformers import BertTokenizer, BertForMaskedLM

sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging

logger = mlogging(loggername="bertMLMtest", logfilename="bertMLMtest")
device = GPU_info()
seed_everything(111)

logfilepath:bertMLMtest_2022-11-23.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#
# - OUTPATH : 출력 모델, vocab 저장할 폴더 경로
#############################################################################################
bisSKKobertModel = 0 # skt/kobert-base-V1 허깅페이스 모델 사용시에는 1로 해줌
model_path = '../../data11/model/bert/mbertV3.0-aihub-NSPMLM-checkout/checkpoint-3211866/'#'klue/bert-base'#'skt/kobert-base-v1'#'jinmang2/kpfbert'#'klue/bert-base'#'bert-base-multilingual-cased'#'beomi/kcbert-base
vocab_path = model_path

do_lower_case_param = 0 # 1=영문인 경우 소문자로만 입력

# tokeniaer 및 model 설정

# skt/kobert 모델은 tokenizer을 XLNET Tokenizer 이므로, 자체 KoBERTTOkenizer 를 불러와서 사용해야 함.
# => 설치 : !pip install 'git+https://github.com/SKTBrain/KOBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# => 출처 : https://velog.io/@m0oon0/KoBERT-%EC%82%AC%EC%9A%A9%EB%B2%95
if bisSKKobertModel == 1:
    from kobert_tokenizer import KoBERTTokenizer
    tokenizer = KoBERTTokenizer.from_pretrained(model_path)
elif do_lower_case_param == 1:
    tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=do_lower_case_param) 
else:
    # strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
    # do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)      
    tokenizer = BertTokenizer.from_pretrained(vocab_path, strip_accents=False, do_lower_case=do_lower_case_param) 
    
model = BertForMaskedLM.from_pretrained(model_path)
model.eval()

Some weights of the model checkpoint at ../../data11/model/bert/mbertV3.0-aihub-NSPMLM-checkout/checkpoint-3211866/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30022, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [3]:
model.num_parameters()

109129798

In [4]:
# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다", "정말 재미있다"))
print(tokenizer.convert_ids_to_tokens(7090))
print(tokenizer.convert_tokens_to_ids('정말'))

30022
[2, 232, 1039, 2339, 1283, 5671, 1057, 1282, 2193, 2173, 1131, 998, 1108, 1094, 1261, 7646, 1058, 610, 1904, 1044, 3, 3300, 6342, 1044, 3]
위해선
3300


In [5]:
#text = ["한국의 수도는 [MASK] 이다"]
#text = ["에펠탑은 [MASK]에 있다"]
text = ['한국의 수도는 [MASK] 이다', 
        '프랑스의 수도는 [MASK] 이다', 
        '2022년 월드컵은 [MASK]에서 열린다',
        '충무공 이순신은 [MASK]에 최고의 장수였다', 
        '배가 아프면 빨리 [MASK]에 가야 한다']

In [6]:
max_length = 128
tokenized_input = tokenizer(text, max_length=max_length, truncation=True, padding='max_length', return_tensors='pt')

In [7]:
print(tokenized_input)

mask_idx_list = []
for tokens in tokenized_input['input_ids'].tolist():
    #print(tokens)
    token_str = [tokenizer.convert_ids_to_tokens(s) for s in tokens]
    print(token_str)
    
    # **위 token_str리스트에서 [MASK] 인덱스를 구함
    # => **해당 [MASK] 안덱스 값 mask_idx 에서는 아래 출력하는데 사용됨
    mask_idx = token_str.index('[MASK]')
    mask_idx_list.append(mask_idx)
    print(f'[MASK] idx : {mask_idx}')
    
print(mask_idx_list)   


{'input_ids': tensor([[    2,  1983,  1131,  2712,  1283,     4,   698,  1044,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [8]:

outputs = model(**tokenized_input)

# 출력은 [1, 256, 143772] 처럼 [batch, token수, vocab수]로 출력되고,
# 여기서 각 toekn수 마다 vocab의 확률이 반환됨.
# => 각 token 중 [MASK] 토근의 vockab이 최대 확률을 구하면 됨
logits = outputs.logits
print(logits.shape)

torch.Size([5, 128, 30022])


In [9]:
for idx, mask_idx in enumerate(mask_idx_list):
    
    # logits_pred 값은 [token수]로 출력됨
    logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
    #print(logits_pred.shape)
  
    # logits에서 [MASK]에 token_id 를 구함
    mask_logits_idx = int(logits_pred[mask_idx])

    # [MASK]에 해당하는 tokeni 구함
    mask_logits_token = tokenizer.convert_ids_to_tokens(mask_logits_idx)

    # 결과 출력 
    print('\n')
    print('*Input: {}'.format(text[idx]))
    print('*[MASK] : {} ({})'.format(mask_logits_token, mask_logits_idx))




*Input: 한국의 수도는 [MASK] 이다
*[MASK] : 일본 (2128)


*Input: 프랑스의 수도는 [MASK] 이다
*[MASK] : 파리 (4846)


*Input: 2022년 월드컵은 [MASK]에서 열린다
*[MASK] : 카타르 (8458)


*Input: 충무공 이순신은 [MASK]에 최고의 장수였다
*[MASK] : 조선 (2760)


*Input: 배가 아프면 빨리 [MASK]에 가야 한다
*[MASK] : 병원 (2289)


/tmp/ipykernel_199315/2607640243.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
